In [1]:
!pip install numpy
!pip install matplotlib

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import datetime
import time

## Binary GWO

### Importing Classes

In [3]:
# sigmoid fun
def sigmoid(x):
  return 1/(1 + np.exp( (-10) * (x - 0.5) ))

In [4]:
class Wolf:

  def __init__(self, dim):

    pos = sigmoid(np.random.rand(dim)) >= 0.5

    # To avoid all zero condition
    while(np.count_nonzero(pos)==0):
      pos = sigmoid(np.random.rand(dim)) >= 0.5

    self.position = pos
    self.obj_score = float('inf')
    self.obj_fun_time = None

Wolf(9).position

array([False,  True,  True, False,  True,  True, False, False, False])

In [5]:
class BinaryGWO:
  
  def __init__(self, dataset, dim, wolves_n, iter_n):
    self.dataset = dataset
    self.dim = dim
    self.wolves_n = wolves_n
    self.iter_n = iter_n

    self.score_history = {
        'alpha' : [],
        'beta' : [],
        'delta' : []
    }

    self.wolves = []
    self.alpha = None
    self.beta = None
    self.delta = None


  def initialize_wolves(self):
    self.wolves = []

    for i in range(self.wolves_n):
      w = Wolf(self.dim)
      #print(w.position)
      self.wolves.append(w)
    
    self.alpha, self.beta, self.delta = self.wolves[:3]
  

  def update_leaders(self):
    for w in self.wolves:

      if w==self.alpha or w==self.beta or w==self.delta:
        continue

      print(f'Position : {w.position}')
      a = datetime.datetime.now()

      ann_model = ANN(w.position, self.dataset)
      ann_model.train()
      w.obj_score = ann_model.test_error()
#     w.obj_score = self.obj_fun(w.position)
#     time.sleep(.2)
      b = datetime.datetime.now()
      w.obj_fun_time = b-a
        
      if w.obj_score <= self.alpha.obj_score:
          self.delta = self.beta
          self.beta = self.alpha
          self.alpha = w
        
      elif w.obj_score <= self.beta.obj_score:
          self.delta = self.beta
          self.beta = w
            
      elif w.obj_score <= self.delta.obj_score:
          self.delta = w
            
    self.score_history['alpha'].append(self.alpha.obj_score)
    self.score_history['beta'].append(self.beta.obj_score)
    self.score_history['delta'].append(self.delta.obj_score)

    print(f'alpha pos: {self.alpha.position}\tscore : {self.alpha.obj_score}')
    print(f'beta pos: {self.beta.position}\tscore : {self.beta.obj_score}')
    print(f'delta pos: {self.delta.position}\tscore : {self.delta.obj_score}')


  def print_wolf_time(self) :
    for w in self.wolves:
        print('Time => ', w.obj_fun_time)

  def update_positions(self, i):

    a = 2 * (1 - (i/self.iter_n))
        
    for w in self.wolves:

      if w==self.alpha or w==self.beta or w==self.delta:
        continue
            
      # r1 & r2 are random vectors in [0, 1]
      r1 = np.random.rand(self.dim)
      r2 = np.random.rand(self.dim)
            
      A1 = a * ((2 * r1) - 1)
      C1 = 2 * r2
          
      D_alpha = abs((C1 * self.alpha.position) - w.position)

      c_step_alpha = sigmoid(A1 * D_alpha)
      b_step_alpha = c_step_alpha >= np.random.rand(self.dim)
      X1 = ((self.alpha.position + b_step_alpha) >= 1)
      #X1 = alpha.position - (A1 * D_alpha)
            
            
      r1 = np.random.rand(self.dim)
      r2 = np.random.rand(self.dim)
            
      A2 = a * ((2 * r1) - 1)
      C2 = 2 * r2

      D_beta = abs((C2 * self.beta.position) - w.position) 

      c_step_beta = sigmoid(A2 * D_beta)
      b_step_beta = c_step_beta >= np.random.rand(self.dim)
      X2 = ((self.beta.position + b_step_beta) >= 1)
      #X2 = beta.position - (A2 * D_beta)
            
            
      r1 = np.random.rand(self.dim)
      r2 = np.random.rand(self.dim)
            
      A3 = a * ((2 * r1) - 1)
      C3 = 2 * r2

      D_delta = abs((C3 * self.delta.position) - w.position)

      c_step_delta = sigmoid(A3 * D_delta)
      b_step_delta = c_step_delta >= np.random.rand(self.dim)
      X3 = ((self.delta.position + b_step_delta) >= 1)
      #X3 = delta.position - (A3 * D_delta)
      
      updated_position = (sigmoid( (X1 + X2 + X3)/3 ) >= np.random.rand(self.dim))

      if np.count_nonzero(updated_position) != 0:
        w.position = updated_position

    

  def optimize(self):

    self.initialize_wolves()

    self.score_history = {
        'alpha' : [],
        'beta' : [],
        'delta' : []
    }

    for i in range(self.iter_n):
      print('\n=> Iteration : ', i)
      self.update_leaders()
      self.update_positions(i)


### ANN Model

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing

import sys

In [7]:
from google.colab import drive
drive.mount("/content/gdrive")


sys.path.insert(0,'/content/gdrive/My Drive/GWO/Model')
sys.path

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


['/content/gdrive/My Drive/GWO/Model',
 '',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [8]:
inFile = "/content/gdrive/My Drive/GWO/Dataset/winequality-red.csv"
data = pd.read_csv(inFile)
data = data.rename(columns={"quality": "target"})
data.head

<bound method NDFrame.head of       fixed acidity  volatile acidity  citric acid  ...  sulphates  alcohol  target
0               7.4             0.700         0.00  ...       0.56      9.4       5
1               7.8             0.880         0.00  ...       0.68      9.8       5
2               7.8             0.760         0.04  ...       0.65      9.8       5
3              11.2             0.280         0.56  ...       0.58      9.8       6
4               7.4             0.700         0.00  ...       0.56      9.4       5
...             ...               ...          ...  ...        ...      ...     ...
1594            6.2             0.600         0.08  ...       0.58     10.5       5
1595            5.9             0.550         0.10  ...       0.76     11.2       6
1596            6.3             0.510         0.13  ...       0.75     11.0       6
1597            5.9             0.645         0.12  ...       0.71     10.2       5
1598            6.0             0.310         

In [9]:
from ANN_Classifier_v2 import *

### Testing Binary GWO

In [10]:
bgwo = BinaryGWO(dataset = data, dim = 11, wolves_n=10, iter_n = 7)

In [ ]:
bgwo.optimize()


=> Iteration :  0
Position : [False False  True False  True False  True False False  True False]
Epoch 1/50
35/35 [==============================] - 1s 3ms/step - loss: 31.4544 - mape: 31.4544
Epoch 2/50
35/35 [==============================] - 0s 3ms/step - loss: 12.5870 - mape: 12.5870
Epoch 3/50
35/35 [==============================] - 0s 3ms/step - loss: 10.3523 - mape: 10.3523
Epoch 4/50
35/35 [==============================] - 0s 3ms/step - loss: 12.3000 - mape: 12.3000
Epoch 5/50
35/35 [==============================] - 0s 4ms/step - loss: 10.1749 - mape: 10.1749
Epoch 6/50
35/35 [==============================] - 0s 4ms/step - loss: 10.0126 - mape: 10.0126
Epoch 7/50
35/35 [==============================] - 0s 4ms/step - loss: 10.3702 - mape: 10.3702
Epoch 8/50
35/35 [==============================] - 0s 3ms/step - loss: 10.1781 - mape: 10.1781
Epoch 9/50
35/35 [==============================] - 0s 3ms/step - loss: 9.9235 - mape: 9.9235
Epoch 10/50
35/35 [=====================